In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy
import random
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import random

In [2]:
df= pd.read_csv("Ranking_expanded.csv",index_col=0)

In [3]:
df=df.dropna()
df

,Text,Content,URL
0,2020 NBA Offseason Preview: Boston Celtics,Hoops Rumors is previewing the 2020 offseason ...,https://t.co/LeUshiYadP
1,2020 NBA Offseason Preview: Boston Celtics,"The situation remained tense in Washington, DC...",https://t.co/3DIQpBgqMH
3,2020 NBA Offseason Preview: Boston Celtics,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur
4,2020 NBA Offseason Preview: Boston Celtics,$20.00+ shipping$12.30+ shipping$17.00+ shippi...,https://t.co/n5xi9wuQ0E
5,2020 NBA Offseason Preview: Boston Celtics,Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9
...,...,...,...
9599,"""A feel for the Game"" Washington Post PS5 Cern...",Join hundreds of Designers and Publishers for ...,https://t.co/XNQdXAPDuR
9600,"""A feel for the Game"" Washington Post PS5 Cern...",Demon's Souls on PS5 has over 180 game help vi...,https://t.co/VH1a41Acds
9601,"""A feel for the Game"" Washington Post PS5 Cern...",Watch Dogs: Legion is officially out! And to c...,https://t.co/Cq9R8HE7OV
9602,"""A feel for the Game"" Washington Post PS5 Cern...",AboutPressCopyrightContact usCreatorsAdvertise...,https://t.co/tINmwUJwDf


In [4]:
df_balanced = pd.read_csv("Sampled_balanced.csv")

In [5]:
df_balanced

,Unnamed: 0,index,ID,Text,Domain,URL,Content,Match
0,0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...,1
1,1,1,1.324130e+18,"In any season, getting sick is tough. But rig...",Medical,https://t.co/OsHSfvef52,AboutPressCopyrightContact usCreatorsAdvertise...,1
2,2,2,1.324130e+18,@FatEmperor And the lastest vicim of the swine...,Medical,https://t.co/HPkErToAHc,"To date, more than 80 cases have been lodged b...",1
3,3,3,1.324130e+18,Fargo Cass Public Health holds additional flu ...,Medical,https://t.co/ztEuaglz07,FARGO — Due to the positive response to last w...,1
4,4,4,1.324130e+18,Kitchener man Hosherdar Polad says he feels di...,Medical,https://t.co/XGEhe89mhn,WATERLOO — Getting a flu shot seemed especiall...,1
...,...,...,...,...,...,...,...,...
23719,23719,1313,1.324440e+18,This Type of Meat Is Linked to Increased Cance...,unknown,https://t.co/2u2KRJSDj9,This Type of Meat Is Linked to Increased Cance...,1
23720,23720,1314,1.324440e+18,@BluebirdsofMO @collhistgarden @HawleyMO @GOP ...,unknown,https://t.co/4vbi3brgeq,Missouri Attorney General Josh Hawley doubled ...,1
23721,23721,1315,1.324440e+18,Quite a mystery!,unknown,https://t.co/vycImaqSZ3,OU Medicine expert looks at why Oklahoma's see...,1
23722,23722,1316,1.324440e+18,I wrote this back in March along with some oth...,unknown,https://t.co/UgrY57yXvP,At the moment I think it’s fair to say that ma...,1


In [6]:
vectorizer_tweet = TfidfVectorizer(max_df = 200,max_features=300)
vectorizer_content = TfidfVectorizer(max_df = 200,max_features=300)
tweet_tfidf = vectorizer_tweet.fit_transform(df_balanced["Text"])
content_tfidf = vectorizer_content.fit_transform(df_balanced["Content"])


In [7]:
test_tweet_tfidf=vectorizer_tweet.transform(df["Text"])
test_content_tfidf = vectorizer_content.transform(df["Content"])

In [8]:
A_tweet_tfidf = tweet_tfidf.A
A_content_tfidf = content_tfidf.A
concat_tfidf =np.hstack((A_tweet_tfidf, A_content_tfidf))

In [9]:
concat_tfidf.shape

(23724, 600)

In [10]:
test_A_tweet_tfidf = test_tweet_tfidf.A
test_A_content_tfidf = test_content_tfidf.A
test_concat_tfidf =np.hstack((test_A_tweet_tfidf, test_A_content_tfidf))

In [11]:
test_concat_tfidf.shape

(9310, 600)

In [12]:
y = df_balanced["Match"]

In [13]:
from sklearn.neural_network import MLPClassifier
model5 = MLPClassifier(random_state=0)
model5.fit(concat_tfidf, y)

MLPClassifier(random_state=0)

In [14]:
y_predict=model5.predict(test_concat_tfidf)

In [15]:
df['prediton']=y_predict

In [16]:
y_prob=model5.predict_proba(test_concat_tfidf)

In [17]:
y_prob[:,1]

array([0.79930742, 0.53205267, 0.71431572, ..., 0.53205267, 0.53205267,
       0.65263656])

In [18]:
df['prob']=y_prob[:,1]

In [19]:
example=df[df['Text']=='2020 NBA Offseason Preview: Boston Celtics'].sort_values(by='prob',ascending=False).head(20)
random_df=df[df['Text']=='2020 NBA Offseason Preview: Boston Celtics'].sample(20,random_state=0)

In [20]:
example=example.reset_index()

example

,index,Text,Content,URL,prediton,prob
0,65,2020 NBA Offseason Preview: Boston Celtics,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
1,33,2020 NBA Offseason Preview: Boston Celtics,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,83,2020 NBA Offseason Preview: Boston Celtics,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
3,23,2020 NBA Offseason Preview: Boston Celtics,I’m a lifelong Dodgers fan and I waited for 32...,https://t.co/VVhmkoXOJL,1,0.833321
4,5,2020 NBA Offseason Preview: Boston Celtics,Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9,1,0.799356
5,0,2020 NBA Offseason Preview: Boston Celtics,Hoops Rumors is previewing the 2020 offseason ...,https://t.co/LeUshiYadP,1,0.799307
6,86,2020 NBA Offseason Preview: Boston Celtics,The PS5 and Xbox Series X graphics are disappo...,https://t.co/zLUuaWkTaZ,1,0.781373
7,71,2020 NBA Offseason Preview: Boston Celtics,Local Player Qualifies for Gran Turismo World ...,https://t.co/b01pNzCH7q,1,0.758134
8,60,2020 NBA Offseason Preview: Boston Celtics,DetailsState Of Origin 2020 live Stream Online...,https://t.co/YWU1wRGUUO,1,0.748858
9,3,2020 NBA Offseason Preview: Boston Celtics,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316


In [21]:
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,3,2020 NBA Offseason Preview: Boston Celtics,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316
1,33,2020 NBA Offseason Preview: Boston Celtics,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,59,2020 NBA Offseason Preview: Boston Celtics,Tara Graham recently emerged from the side ent...,https://t.co/rCJiP7lvN5,1,0.532053
3,17,2020 NBA Offseason Preview: Boston Celtics,"Updated: 11/4/2020As of right now, I do plan o...",https://t.co/H7rJA0iJKR,0,0.000392
4,14,2020 NBA Offseason Preview: Boston Celtics,Nicki Minaj took to social media on Tuesday (N...,https://t.co/VzU1PmSPj4,1,0.689573
5,64,2020 NBA Offseason Preview: Boston Celtics,Photograph by ShutterstockSplit ScreenOlivia S...,https://t.co/japetnR2NV,1,0.532053
6,65,2020 NBA Offseason Preview: Boston Celtics,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
7,83,2020 NBA Offseason Preview: Boston Celtics,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
8,36,2020 NBA Offseason Preview: Boston Celtics,"Thu, Nov 05 | ZoomHow to Boost Your Immune Sys...",https://t.co/oYavNCAjTG,1,0.532053
9,82,2020 NBA Offseason Preview: Boston Celtics,A feel for the gameSony’s big PS5 bet is that ...,https://t.co/A9nmkniGDy,0,0.119481


In [22]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[0,0,0,0,2,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0]
len(trues)
randoms=np.random.random((20))
randoms

array([0.98985996, 0.13836918, 0.82805264, 0.6496048 , 0.61338185,
       0.24080954, 0.04989047, 0.83151661, 0.55174963, 0.59156259,
       0.74928054, 0.13646877, 0.37921089, 0.10776646, 0.32691827,
       0.41411576, 0.55162432, 0.2334219 , 0.62471215, 0.32336749])

In [23]:
from sklearn.metrics import ndcg_score
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

0.49371539472324644

In [24]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.30412582937211724

In [26]:
randoms=np.random.random((20))
random_score=np.asarray([randoms])
ndcg_score(true_relevance, random_score)

0.38878797595646547

In [27]:
df_1=pd.read_csv("Ranking.csv")

In [28]:
example=df[df['Text']==df_1['Text'][3]].sort_values(by='prob',ascending=False).head(20)
example=example.reset_index()
example

,index,Text,Content,URL,prediton,prob
0,359,Congrats to user davrogbro62 for winning the t...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
1,327,Congrats to user davrogbro62 for winning the t...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,377,Congrats to user davrogbro62 for winning the t...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
3,317,Congrats to user davrogbro62 for winning the t...,I’m a lifelong Dodgers fan and I waited for 32...,https://t.co/VVhmkoXOJL,1,0.833321
4,299,Congrats to user davrogbro62 for winning the t...,Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9,1,0.799356
5,294,Congrats to user davrogbro62 for winning the t...,Hoops Rumors is previewing the 2020 offseason ...,https://t.co/LeUshiYadP,1,0.799307
6,380,Congrats to user davrogbro62 for winning the t...,The PS5 and Xbox Series X graphics are disappo...,https://t.co/zLUuaWkTaZ,1,0.781373
7,365,Congrats to user davrogbro62 for winning the t...,Local Player Qualifies for Gran Turismo World ...,https://t.co/b01pNzCH7q,1,0.758134
8,354,Congrats to user davrogbro62 for winning the t...,DetailsState Of Origin 2020 live Stream Online...,https://t.co/YWU1wRGUUO,1,0.748858
9,297,Congrats to user davrogbro62 for winning the t...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316


In [33]:
random_df=df[df['Text']==df_1['Text'][3]].sample(20,random_state=0)
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,297,Congrats to user davrogbro62 for winning the t...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316
1,327,Congrats to user davrogbro62 for winning the t...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,353,Congrats to user davrogbro62 for winning the t...,Tara Graham recently emerged from the side ent...,https://t.co/rCJiP7lvN5,1,0.532053
3,311,Congrats to user davrogbro62 for winning the t...,"Updated: 11/4/2020As of right now, I do plan o...",https://t.co/H7rJA0iJKR,0,0.000392
4,308,Congrats to user davrogbro62 for winning the t...,Nicki Minaj took to social media on Tuesday (N...,https://t.co/VzU1PmSPj4,1,0.689573
5,358,Congrats to user davrogbro62 for winning the t...,Photograph by ShutterstockSplit ScreenOlivia S...,https://t.co/japetnR2NV,1,0.532053
6,359,Congrats to user davrogbro62 for winning the t...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
7,377,Congrats to user davrogbro62 for winning the t...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
8,330,Congrats to user davrogbro62 for winning the t...,"Thu, Nov 05 | ZoomHow to Boost Your Immune Sys...",https://t.co/oYavNCAjTG,1,0.532053
9,376,Congrats to user davrogbro62 for winning the t...,A feel for the gameSony’s big PS5 bet is that ...,https://t.co/A9nmkniGDy,0,0.119481


In [29]:
df_1['Text'][3]

'Congrats to user davrogbro62 for winning the title in the latest Random Players at #pennantchase'

In [32]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[0,0,0,2,0,0,0,0,0,2,2,0,0,0,2,0,0,0,0,0]
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

0.5468122521288148

In [34]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.3815380271520173

In [35]:
example=df[df['Text']==df_1['Text'][4]].sort_values(by='prob',ascending=False).head(20)
example=example.reset_index()
example

,index,Text,Content,URL,prediton,prob
0,396,Check out 1972 Topps Baseball Lot @eBay@sport...,$20.00+ shipping$12.30+ shipping$17.00+ shippi...,https://t.co/n5xi9wuQ0E,1,0.999495
1,475,Check out 1972 Topps Baseball Lot @eBay@sport...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,0,0.018699
2,481,Check out 1972 Topps Baseball Lot @eBay@sport...,Some parts of this page is temporarily unavail...,https://t.co/GMHbLxztEy,0,0.014834
3,448,Check out 1972 Topps Baseball Lot @eBay@sport...,Flu Guide: What to Know This Season7 Risk Fact...,https://t.co/3C8bUXscnp,0,0.010151
4,434,Check out 1972 Topps Baseball Lot @eBay@sport...,Killer flu outbreak is to blame for a 42% spik...,https://t.co/3Xtil1alvm,0,0.008306
5,436,Check out 1972 Topps Baseball Lot @eBay@sport...,- Face masks might help protect against the fl...,https://t.co/hHY6U9ZPir,0,0.003039
6,473,Check out 1972 Topps Baseball Lot @eBay@sport...,Gaming technology improves with every new gene...,https://t.co/qvaMjll3ha,0,0.001119
7,425,Check out 1972 Topps Baseball Lot @eBay@sport...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,0,0.000977
8,432,Check out 1972 Topps Baseball Lot @eBay@sport...,A case of a rare variant of Influenza A (H1N2)...,https://t.co/yozi9tamCc,0,0.000859
9,460,Check out 1972 Topps Baseball Lot @eBay@sport...,AboutPressCopyrightContact usCreatorsAdvertise...,https://t.co/G3nXtFyB4z,0,0.000346


In [36]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

1.0

In [37]:
random_df=df[df['Text']==df_1['Text'][4]].sample(20,random_state=0)
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,395,Check out 1972 Topps Baseball Lot @eBay@sport...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,0,0.000041
1,425,Check out 1972 Topps Baseball Lot @eBay@sport...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,0,0.000977
2,451,Check out 1972 Topps Baseball Lot @eBay@sport...,Tara Graham recently emerged from the side ent...,https://t.co/rCJiP7lvN5,0,0.000175
3,409,Check out 1972 Topps Baseball Lot @eBay@sport...,"Updated: 11/4/2020As of right now, I do plan o...",https://t.co/H7rJA0iJKR,0,0.000006
4,406,Check out 1972 Topps Baseball Lot @eBay@sport...,Nicki Minaj took to social media on Tuesday (N...,https://t.co/VzU1PmSPj4,0,0.000076
5,456,Check out 1972 Topps Baseball Lot @eBay@sport...,Photograph by ShutterstockSplit ScreenOlivia S...,https://t.co/japetnR2NV,0,0.000175
6,457,Check out 1972 Topps Baseball Lot @eBay@sport...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,0,0.000067
7,475,Check out 1972 Topps Baseball Lot @eBay@sport...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,0,0.018699
8,428,Check out 1972 Topps Baseball Lot @eBay@sport...,"Thu, Nov 05 | ZoomHow to Boost Your Immune Sys...",https://t.co/oYavNCAjTG,0,0.000175
9,474,Check out 1972 Topps Baseball Lot @eBay@sport...,A feel for the gameSony’s big PS5 bet is that ...,https://t.co/A9nmkniGDy,0,0.000045


In [38]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.23981246656813138

In [40]:
example=df[df['Text']==df_1['Text'][5]].sort_values(by='prob',ascending=False).head(20)
example=example.reset_index()
example

,index,Text,Content,URL,prediton,prob
0,555,LeBron James rompe la red para hundir a Stephe...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
1,523,LeBron James rompe la red para hundir a Stephe...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,573,LeBron James rompe la red para hundir a Stephe...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
3,513,LeBron James rompe la red para hundir a Stephe...,I’m a lifelong Dodgers fan and I waited for 32...,https://t.co/VVhmkoXOJL,1,0.833321
4,495,LeBron James rompe la red para hundir a Stephe...,Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9,1,0.799356
5,490,LeBron James rompe la red para hundir a Stephe...,Hoops Rumors is previewing the 2020 offseason ...,https://t.co/LeUshiYadP,1,0.799307
6,576,LeBron James rompe la red para hundir a Stephe...,The PS5 and Xbox Series X graphics are disappo...,https://t.co/zLUuaWkTaZ,1,0.781373
7,561,LeBron James rompe la red para hundir a Stephe...,Local Player Qualifies for Gran Turismo World ...,https://t.co/b01pNzCH7q,1,0.758134
8,550,LeBron James rompe la red para hundir a Stephe...,DetailsState Of Origin 2020 live Stream Online...,https://t.co/YWU1wRGUUO,1,0.748858
9,493,LeBron James rompe la red para hundir a Stephe...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316


In [43]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[0,0,0,0,2,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

0.49371539472324644

In [44]:
random_df=df[df['Text']==df_1['Text'][5]].sample(20,random_state=0)
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,493,LeBron James rompe la red para hundir a Stephe...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316
1,523,LeBron James rompe la red para hundir a Stephe...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,549,LeBron James rompe la red para hundir a Stephe...,Tara Graham recently emerged from the side ent...,https://t.co/rCJiP7lvN5,1,0.532053
3,507,LeBron James rompe la red para hundir a Stephe...,"Updated: 11/4/2020As of right now, I do plan o...",https://t.co/H7rJA0iJKR,0,0.000392
4,504,LeBron James rompe la red para hundir a Stephe...,Nicki Minaj took to social media on Tuesday (N...,https://t.co/VzU1PmSPj4,1,0.689573
5,554,LeBron James rompe la red para hundir a Stephe...,Photograph by ShutterstockSplit ScreenOlivia S...,https://t.co/japetnR2NV,1,0.532053
6,555,LeBron James rompe la red para hundir a Stephe...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
7,573,LeBron James rompe la red para hundir a Stephe...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
8,526,LeBron James rompe la red para hundir a Stephe...,"Thu, Nov 05 | ZoomHow to Boost Your Immune Sys...",https://t.co/oYavNCAjTG,1,0.532053
9,572,LeBron James rompe la red para hundir a Stephe...,A feel for the gameSony’s big PS5 bet is that ...,https://t.co/A9nmkniGDy,0,0.119481


In [45]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.30412582937211724

In [47]:
example=df[df['Text']==df_1['Text'][50]].sort_values(by='prob',ascending=False).head(20)
example=example.reset_index()
example

,index,Text,Content,URL,prediton,prob
0,4965,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,1,0.915982
1,4933,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,1,0.915922
2,4983,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,NES & Super NES - September Game Updates - Nin...,https://t.co/HXQR7ha0aw,1,0.859013
3,4923,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,I’m a lifelong Dodgers fan and I waited for 32...,https://t.co/VVhmkoXOJL,1,0.833321
4,4905,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9,1,0.799356
5,4900,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Hoops Rumors is previewing the 2020 offseason ...,https://t.co/LeUshiYadP,1,0.799307
6,4986,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,The PS5 and Xbox Series X graphics are disappo...,https://t.co/zLUuaWkTaZ,1,0.781373
7,4971,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Local Player Qualifies for Gran Turismo World ...,https://t.co/b01pNzCH7q,1,0.758134
8,4960,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,DetailsState Of Origin 2020 live Stream Online...,https://t.co/YWU1wRGUUO,1,0.748858
9,4903,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Features of PennantChase.comMost users play in...,https://t.co/xLqL4PMYur,1,0.714316


In [50]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,2,0]
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

0.535888772338792

In [53]:
random_df=df[df['Text']==df_1['Text'][50]].sample(20,random_state=1)
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,4943,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Boy who said he developed narcolepsy after swi...,https://t.co/KgMuOzKURk,1,0.532053
1,4934,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Let’s Learn From South Korea’s Aggressive Roll...,https://t.co/pdmTl7TE27,0,0.000877
2,4949,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,"In the midst of the coronavirus pandemic, heal...",https://t.co/rm7hZwcMBr,1,0.532053
3,4961,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,"||||||||Game, Smyths and Argos will remain ope...",https://t.co/wdKeo0T8Iq,1,0.532053
4,4980,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,"NBA 2K21: ""Everything is Game"" Launch SpotNo h...",https://t.co/1v0ta9JYsr,1,0.532053
5,4952,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,"Health Care HeadlinesThu Nov 12, 2020 1:08 PM ...",https://t.co/OuPAaUQGcS,0,0.010771
6,4990,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,"Sign in to add this item to your wishlist, fol...",https://t.co/Xa4iCOo8VV,1,0.532053
7,4947,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Major flu outbreak could 'overwhelm' the NHS t...,https://t.co/ytI4l386Va,1,0.532053
8,4991,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,AboutPressCopyrightContact usCreatorsAdvertise...,https://t.co/LZ0hOog6yE,1,0.532053
9,4993,💔IDAHO HAS FIRST 2 FLU DEATHS💔Both also positi...,Join hundreds of Designers and Publishers for ...,https://t.co/XNQdXAPDuR,0,0.470729


In [54]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.5153897772890079

In [56]:
example=df[df['Text']==df_1['Text'][81]].sort_values(by='prob',ascending=False).head(20)
example=example.reset_index()
example

,index,Text,Content,URL,prediton,prob
0,8019,"New post: ""Here’s every game coming to PlaySta...",Gaming technology improves with every new gene...,https://t.co/qvaMjll3ha,1,0.975398
1,7978,"New post: ""Here’s every game coming to PlaySta...",A case of a rare variant of Influenza A (H1N2)...,https://t.co/yozi9tamCc,0,0.004048
2,7982,"New post: ""Here’s every game coming to PlaySta...",- Face masks might help protect against the fl...,https://t.co/hHY6U9ZPir,0,0.003450
3,7971,"New post: ""Here’s every game coming to PlaySta...",And it's not just Ontario. Long lines and shor...,https://t.co/gWe1kColu5,0,0.002160
4,7952,"New post: ""Here’s every game coming to PlaySta...",Nicki Minaj took to social media on Tuesday (N...,https://t.co/VzU1PmSPj4,0,0.001109
5,7980,"New post: ""Here’s every game coming to PlaySta...",Killer flu outbreak is to blame for a 42% spik...,https://t.co/3Xtil1alvm,0,0.000721
6,7957,"New post: ""Here’s every game coming to PlaySta...",Bruce Maxwell29-Year-Old2020 StatsAVG.000HR0RB...,https://t.co/6E0Jbes6WU,0,0.000659
7,7943,"New post: ""Here’s every game coming to PlaySta...",Las elecciones de Estados Unidos han sido lo m...,https://t.co/WKG5S91gJ9,0,0.000590
8,8013,"New post: ""Here’s every game coming to PlaySta...","MFox.live-Watch Live SportsEvents of Thursday,...",https://t.co/fiY0gkMvcX,0,0.000557
9,8003,"New post: ""Here’s every game coming to PlaySta...",Why is it so awesome?Whiteboard.fi is a simple...,https://t.co/qvqJyDwkPc,0,0.000480


In [58]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues=[2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues])
scores = np.asarray([example['prob']])
ndcg_score(true_relevance, scores)

0.8629033714585109

In [59]:
random_df=df[df['Text']==df_1['Text'][81]].sample(20,random_state=1)
random_df=random_df.reset_index()
random_df

,index,Text,Content,URL,prediton,prob
0,7981,"New post: ""Here’s every game coming to PlaySta...",Boy who said he developed narcolepsy after swi...,https://t.co/KgMuOzKURk,0,0.000067
1,7972,"New post: ""Here’s every game coming to PlaySta...",Let’s Learn From South Korea’s Aggressive Roll...,https://t.co/pdmTl7TE27,0,0.000002
2,7987,"New post: ""Here’s every game coming to PlaySta...","In the midst of the coronavirus pandemic, heal...",https://t.co/rm7hZwcMBr,0,0.000067
3,7999,"New post: ""Here’s every game coming to PlaySta...","||||||||Game, Smyths and Argos will remain ope...",https://t.co/wdKeo0T8Iq,0,0.000067
4,8018,"New post: ""Here’s every game coming to PlaySta...","NBA 2K21: ""Everything is Game"" Launch SpotNo h...",https://t.co/1v0ta9JYsr,0,0.000067
5,7990,"New post: ""Here’s every game coming to PlaySta...","Health Care HeadlinesThu Nov 12, 2020 1:08 PM ...",https://t.co/OuPAaUQGcS,0,0.000259
6,8028,"New post: ""Here’s every game coming to PlaySta...","Sign in to add this item to your wishlist, fol...",https://t.co/Xa4iCOo8VV,0,0.000067
7,7985,"New post: ""Here’s every game coming to PlaySta...",Major flu outbreak could 'overwhelm' the NHS t...,https://t.co/ytI4l386Va,0,0.000067
8,8029,"New post: ""Here’s every game coming to PlaySta...",AboutPressCopyrightContact usCreatorsAdvertise...,https://t.co/LZ0hOog6yE,0,0.000067
9,8031,"New post: ""Here’s every game coming to PlaySta...",Join hundreds of Designers and Publishers for ...,https://t.co/XNQdXAPDuR,0,0.000028


In [63]:
indes=[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9]
trues_r=[0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
true_relevance = np.asarray([trues_r])
scores = np.asarray([random_df['prob']])
ndcg_score(true_relevance, scores)

0.3544771804889836